# Stock Exchange Example

## Install Requirements

In [4]:
!pip install --quiet agent-exchange==0.0.4 pandas numpy

In [1]:
from random import shuffle, random, randint
from typing import Sequence, Tuple, Dict, Union

import numpy as np
import pandas as pd
from agent_exchange.agent import Agent
from agent_exchange.exchange import Exchange

from stock_exchange_v1 import * # requires stock_exchange_v1.py file to be local
from stock_agent_v1 import *    # requires stock_agent_v1.py file to be local

## Make Stock Exchange Agents

In [2]:
class StockAgentV1NaiveMaker(StockAgentV1):
    """Naive agent that acts as a market maker.
    If there is a spread, this agent will place
    an order on the buy side 1/2 of the time, on
    the sell side 1/2 of the time.

    Note: in V1, there is no way to place multiple
    orders within the same time step.
    """
    def __init__(self, initial_num_shares=1000, initial_capital=100000):
        super().__init__(initial_num_shares, initial_capital)

    def get_action(self, order_book: StockExchangeV1OrderBook):
        if random() < .5: # buy just over current bid
            buy_price = order_book.get_bid() + 0.01 # penny-up on the market-clearing buy price
            
            # Randomly buy as little as 0 or as much as we can
            buy_amount = randint(0, 1000) # limit buy

            order = StockExchangeV1Action(StockExchangeV1OrderTypes.LIMIT, buy_amount, buy_price)

            return order
        else: # sell just under current ask
            sell_price = order_book.get_ask() - 0.01

            # Randomly sell as little as 0 and as much as 10
            sell_amount = randint(0, 1000)

            order = StockExchangeV1Action(StockExchangeV1OrderTypes.LIMIT, -sell_amount, sell_price)

            return order


class StockAgentV1NaiveTaker(StockAgentV1):
    """Naive agent that acts as a liquidity
    taker. This agent speculates by placing
    market orders of buy 1/2 of the time and
    sell 1/2 of the time. Here we ignore
    constraints on short selling, allowing
    agents to shorts sell without limit.

    Also, if the taker's order exhausts the
    order book, then only the portion of their
    order in the order book gets filled.
    """
    def __init__(self, initial_num_shares=1000, initial_capital=100000):
        super().__init__(initial_num_shares, initial_capital)

    def get_action(self, order_book: StockExchangeV1OrderBook):
        if random() < .5: # buy
            expected_buy_price = order_book.get_ask()
            
            # Randomly decide on how much to buy
            max_buy_amount = self.internal_state.get_capital() // expected_buy_price
            num_shares_to_buy = randint(0, max_buy_amount)

            return StockExchangeV1Action(StockExchangeV1OrderTypes.MARKET, num_shares_to_buy)

        else:
            expected_sell_price = order_book.get_bid()

            # Randomly decide on how much to sell
            max_sell_amount = self.internal_state.get_num_shares()
            num_shares_to_sell = randint(0, max_sell_amount)

            return StockExchangeV1Action(StockExchangeV1OrderTypes.MARKET, -num_shares_to_sell)

## Run simulations

In [3]:
# Run a simple 
NMAKER, NTAKER, NSTEPS = 10, 1, 100
agents = [StockAgentV1NaiveMaker() for _ in range(NMAKER)]
agents += [StockAgentV1NaiveTaker() for _ in range(NTAKER)]
exchange = StockExchangeV1(agents)

exchange.simulate_steps(NSTEPS)
# print(exchange.order_book)

In [4]:
agents[-1]

{'internal_state': {'num_shares': [1000, 1590, 1786, 1846, 1199, 1881, 1966, 1971, 1856, 1916, 1921, 1967, 1995, 1188, 337, 1616, 1511, 1996, 1440, 187, 605, 1352, 1944, 1195, 1270, 1426, 1588, 1806, 1924, 135, 1804, 1910, 1928, 1928, 1955, 1957, 140, 197, 5, 1189, 386, 1240, 652, 98, 117, 10, 1, 1, 1, 1, 0, 0, 520, 1807, 913, 322, 1384, 1687, 1882, 1902, 1975, 1329, 1909, 1498, 1502, 696, 1838, 1544, 1117, 1582, 1245, 352, 1978, 1984, 1987, 1375, 1704, 1499, 992, 1987, 1987, 1989, 1994, 1994, 1730, 1749, 1749, 1873, 1952, 1952, 1952, 1952, 1952, 1972, 1977, 1984, 1990, 1990, 1990, 1996, 1996], 'capital': [100000, 40976.4, 21370.52, 15369.32, 80062.85, 11862.850000000006, 3362.850000000006, 2862.900000000006, 14361.750000000007, 8361.750000000007, 7861.8000000000075, 3262.2600000000075, 462.8200000000079, 81130.54000000001, 166196.5, 38284.610000000015, 48782.51000000002, 282.5100000000166, 55875.07000000002, 181162.54000000004, 139362.54000000004, 64669.880000000034, 5481.720000000037